<a href="https://colab.research.google.com/github/SemihAkkoc/machine_learning/blob/main/flowers_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd

In [2]:
COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setose', 'Versicolor', 'Virginica']

train = pd.read_csv('https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv', names=COLUMN_NAMES, header=0)
test = pd.read_csv('https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv', names=COLUMN_NAMES, header=0)
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [3]:
train_y = train.pop('Species')
test_y = test.pop('Species')

In [4]:
# creating the input function

def input_fn(features, labels, training=True, batch_size=256):
    ds = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    if training:
        ds = ds.shuffle(1000).repeat()
    return ds.batch(batch_size)

In [5]:
# feature columns

feature_columns = []
for key in train.keys():
    feature_columns.append(tf.feature_column.numeric_column(key=key))

In [6]:
# creating our classifier model

model = tf.estimator.DNNClassifier(feature_columns=feature_columns, hidden_units=[30, 10], n_classes=3)

"""
feature_columns is our feature columns that our model will going to learn from such as petal length etc.
hidden_units is hidden layers and we created 30 and 10 nodes respectively
n_classes is the number of classes we are going to choose from since we have 3 different species of flower we send 3
"""

In [7]:
# training our model
model.train(input_fn=lambda: input_fn(train, train_y), steps=5000)

In [8]:
# evaluating model
result = model.evaluate(input_fn=lambda: input_fn(test, test_y, training=False))

In [9]:
# printing the accuracy
print(result['accuracy'])

0.73333335


In [10]:
# inputing our values and making model predict

def predict_fn(COLUMN_NAMES=COLUMN_NAMES, model=model):

    def input_fn_without_labels(features, batch_size=256):
        return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

    features = COLUMN_NAMES[:-1]
    predict = {}
    
    print('Please type numeric values as prompted.')
    for feature in features:
        val = input(feature + ': ')
        predict[feature] = [float(val)]
    
    # model is making prediction based on our values
    predictions = model.predict(input_fn=lambda: input_fn_without_labels(predict))

    for key in predictions:
        # print(key)
        class_id = key['class_ids'][0]
        probability = key['probabilities'][class_id]

        print(f'\nPrediction is {SPECIES[class_id]} {probability*100}')

```
Example:
                Setosa   Versicolor   Virginica
Sepal Length:    5.1        5.9          6.9
Sepal  Witdh:    3.3        3.0          3.1
Petal Length:    1.7        4.2          5.4
Petal  Witdh:    0.5        1.5          2.1
```



In [11]:
predict_fn()

Please type numeric values as prompted.
SepalLength: 5.1
SepalWidth: 3.3
PetalLength: 1.7
PetalWidth: 0.5

Prediction is Setose 88.1507158279419
